<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_pgd_CNN_SHAP_detection_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 17.3 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [6]:
pgd_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/24_feature/pgd/pgd_cnn_bin_feature'

In [7]:
pgd_df = pd.read_csv(pgd_path)

In [8]:
pgd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

# Load DL-based IDS model

In [9]:
model_cnn = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_cnn_bin.h5', compile=True)

# SHAP

In [10]:
explainer = shap.KernelExplainer(model_cnn, shap.sample(df, 100))

In [11]:
pgd_100 = pgd_df.head(100)

42m ~ 1g

In [12]:
shap_values = explainer.shap_values(pgd_100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
len(shap_values)

1

In [14]:
type(shap_values[0])

numpy.ndarray

In [15]:
len(shap_values[0])

100

In [16]:
shap_values[0].shape

(100, 76)

In [17]:
data_df = pd.DataFrame(shap_values[0])

In [18]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,-0.000442,-0.000156,0.000071,-0.000109,0.000138,0.000000,0.000000,0.000050
1,0.000024,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000181,0.000073,0.000017,0.000000,0.000095
2,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000047,0.0,...,0.0,0.0,-0.000779,0.000000,0.000000,0.000029,0.000071,0.000000,0.000087,0.000076
3,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000050,-0.000232,0.000107,0.000103,-0.000244,0.000000,0.000029
4,-0.000635,0.0,0.0,0.000102,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,-0.000260,0.000000,0.000000,0.000000,-0.000824,0.000000,0.000166,-0.000233


In [19]:
data_df.columns = df.columns
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      100 non-null    float64
 1   Tot Fwd Pkts       100 non-null    float64
 2   Tot Bwd Pkts       100 non-null    float64
 3   TotLen Fwd Pkts    100 non-null    float64
 4   TotLen Bwd Pkts    100 non-null    float64
 5   Fwd Pkt Len Max    100 non-null    float64
 6   Fwd Pkt Len Min    100 non-null    float64
 7   Fwd Pkt Len Mean   100 non-null    float64
 8   Fwd Pkt Len Std    100 non-null    float64
 9   Bwd Pkt Len Max    100 non-null    float64
 10  Bwd Pkt Len Min    100 non-null    float64
 11  Bwd Pkt Len Mean   100 non-null    float64
 12  Bwd Pkt Len Std    100 non-null    float64
 13  Flow Byts/s        100 non-null    float64
 14  Flow Pkts/s        100 non-null    float64
 15  Flow IAT Mean      100 non-null    float64
 16  Flow IAT Std       100 non-

In [20]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/CNN_HSJ/pgd_100_samples_[0].csv', index=False)

In [21]:
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,-0.000442,-0.000156,0.000071,-0.000109,0.000138,0.000000,0.000000,0.000050
1,0.000024,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000181,0.000073,0.000017,0.000000,0.000095
2,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000047,0.0,...,0.0,0.0,-0.000779,0.000000,0.000000,0.000029,0.000071,0.000000,0.000087,0.000076
3,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000050,-0.000232,0.000107,0.000103,-0.000244,0.000000,0.000029
4,-0.000635,0.0,0.0,0.000102,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,-0.000260,0.000000,0.000000,0.000000,-0.000824,0.000000,0.000166,-0.000233


In [22]:
data_dict = {}
for idx in range(0, 100):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [23]:
print(data_dict)

{'Adv sample 0': ['ACK Flag Cnt', 'SYN Flag Cnt', 'FIN Flag Cnt', 'Bwd Pkts/s', 'Bwd PSH Flags', 'Fwd Pkts/b Avg', 'Fwd IAT Max', 'Init Bwd Win Byts', 'URG Flag Cnt', 'PSH Flag Cnt'], 'Adv sample 1': ['Fwd IAT Max', 'FIN Flag Cnt', 'Bwd Pkts/s', 'Fwd Pkts/b Avg', 'Bwd PSH Flags', 'PSH Flag Cnt', 'Bwd IAT Max', 'Bwd IAT Min', 'Active Min', 'ACK Flag Cnt'], 'Adv sample 2': ['FIN Flag Cnt', 'Fwd IAT Max', 'ACK Flag Cnt', 'Bwd Pkts/s', 'Bwd IAT Max', 'Bwd Blk Rate Avg', 'Bwd PSH Flags', 'PSH Flag Cnt', 'Bwd IAT Tot', 'Fwd IAT Mean'], 'Adv sample 3': ['FIN Flag Cnt', 'Bwd Pkts/s', 'Fwd IAT Max', 'ACK Flag Cnt', 'Fwd Pkts/b Avg', 'Bwd PSH Flags', 'PSH Flag Cnt', 'URG Flag Cnt', 'Active Min', 'Init Bwd Win Byts'], 'Adv sample 4': ['Down/Up Ratio', 'SYN Flag Cnt', 'Fwd IAT Mean', 'Bwd Pkts/s', 'PSH Flag Cnt', 'Idle Max', 'Fwd Blk Rate Avg', 'URG Flag Cnt', 'Bwd PSH Flags', 'TotLen Fwd Pkts'], 'Adv sample 5': ['ACK Flag Cnt', 'SYN Flag Cnt', 'FIN Flag Cnt', 'Bwd PSH Flags', 'Bwd IAT Tot', 'Bwd 

In [24]:
len(shap_values[0])

100

In [25]:
len(data_dict)

100

# Detection phase

In [26]:
import os
import pandas as pd

In [27]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/cnn_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,URG Flag Cnt,149
1,Bwd Pkts/s,148
2,FIN Flag Cnt,141
3,Bwd PSH Flags,131
4,SYN Flag Cnt,123
5,PSH Flag Cnt,116
6,ACK Flag Cnt,98
7,Down/Up Ratio,97
8,Flow Pkts/s,86
9,Init Bwd Win Byts,82


## Detection rate (top 24) = 92/100

In [28]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max']


In [29]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:92


## Detection rate (top 38) = 8/100

In [30]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['URG Flag Cnt', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Bwd PSH Flags', 'SYN Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'Down/Up Ratio', 'Flow Pkts/s', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Bwd URG Flags', 'Flow Duration', 'Idle Min', 'Bwd Pkt Len Max', 'Fwd IAT Max', 'Flow IAT Std', 'Fwd IAT Mean', 'Flow IAT Max', 'Tot Fwd Pkts', 'Bwd Pkt Len Mean', 'Bwd IAT Std', 'Idle Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'Idle Mean', 'Bwd Seg Size Avg', 'Pkt Len Mean', 'Flow IAT Mean', 'Pkt Len Std', 'TotLen Bwd Pkts', 'Bwd Pkt Len Std', 'Bwd Header Len', 'Subflow Bwd Pkts', 'Fwd Act Data Pkts', 'Fwd Pkt Len Min', 'Pkt Len Min']


In [31]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:89
